# Uncertainties 

Since most exoplanet properties are derived from measurements, most have uncertainties. The reliability of a visualization or calculation depends crucially on at least a qualitative understanding of the uncertainties associed with each quantity. To try to help with this, `exoatlas` provides an interface to both uncertainties that are reported in original archive tables and propagated uncertainties estimated for calculated quantities.  

In [ ]:
import exoatlas as ea
import astropy.units as u

ea.version()

This page expands the very brief discussion of uncertainties on (Populations)[populations.ipynb], providing more details and a little explanation of how uncertainty estimates are calculated. 

In [ ]:
pop = ea.TransitingExoplanets()

## How do we retrieve uncertainties? 

We will often want to know the uncertainty on a particular quantity. We can retrieve this either with the `.get_uncertainty()` method, or by appending `_uncertainty` to the name of a quantity. For core table quantities, uncertainties are extracted directly from the table. 


In [ ]:
sigma = pop.get_uncertainty('radius')
sigma

In [ ]:
sigma = pop.radius_uncertainty()
sigma

Some uncertainties might be asymmetric, with different upper and lower uncertainties, such as 
$x^{+\sigma_{upper}}_{-\sigma_{lower}}$. We can extract these asymmetric uncertainties with `.get_uncertainty_lowerupper()` or by appending `_uncertainty_lowerupper`.

In [ ]:
sigma_lower, sigma_upper = pop.get_uncertainty_lowerupper('stellar_teff')
sigma_lower, sigma_upper

In [ ]:
sigma_lower, sigma_upper = pop.stellar_teff_uncertainty_lowerupper()
sigma_lower, sigma_upper

We can force asymmetric uncertaintoies to be symmetric, calculated as $\sigma = (\sigma_{lower} + \sigma_{upper})/2$, just by asking for the a simple symmetric uncertainty.

In [ ]:
sigma = pop.get_uncertainty('stellar_teff')
sigma 

We can also estimate uncertainties on derived quantities in the same way. Behind the scenes, uncertainties on derived quantities are estimated using [`astropy.uncertainty`](https://docs.astropy.org/en/stable/uncertainty/index.html). Samples are created for each ingredient table column using skew-normal distributions for asymmetric uncertainties as advocated by [Pineda et al. (2021)](https://ui.adsabs.harvard.edu/abs/2021ApJ...918...40P/abstract), and estimated errors are based on the central 68% confidence intervals of the calculated distributions. 


In [ ]:
pop.get_uncertainty('scale_height')

In [ ]:
pop.scale_height_uncertainty()

We might commonly be interested in the fractional uncertainty on a quantity. We can either calculate this ourselves, or use the `.get_fractional_uncertainty` wrapper.

In [ ]:
pop.get_uncertainty('scale_height')/pop.get('scale_height')

In [ ]:
pop.get_fractional_uncertainty('scale_height')

Keyword arguments can be supplied when calculating derived quantities, to be passed into the function that actually does the calculating. 

In [ ]:
pop.teq_uncertainty(albedo=0.5)

In [ ]:
pop.get_uncertainty('teq', albedo=0.5)

In [ ]:
pop.teq_uncertainty_lowerupper(albedo=0.5)

In [ ]:
pop.get_uncertainty_lowerupper('teq', albedo=0.5)

## How do we get more precise propagated uncertainties? 

Propagated uncertainties are calculated by generating lots of numerical samples for each quantity for each planet, calculating derived quantities, and then estimated confidence intervals from the calculated samples. To avoid memory issues on larger planet populations, the default number of samples to use for these distributions is $\sf N_{samples}=100$. That is not enough to achieve precise uncertainty estimates, so in practice we loop over $\sf N_{iteration}$ iterations calculating uncertainties, and average the results together. We target a desired fractional uncertainty on the uncertainties $\sf f$ by noting that $\sf f \approx \sqrt{1/N_{total}}$, where $\sf N_{total} = N_{samples}\cdot N_{iteration}$ is effectively the total number of samples we generate. By default, we target $\sf f = 0.05$, so $\sf N_{iterations} = 4$ iterations are needed. 

In this example, we'll look in detail at the uncertainties for a small subset population. We'll crudely estimate the fractional uncertainty on the uncertainties by doing two independent calculations and looking at their difference.

In [ ]:
subset = pop[:5]

In [ ]:
subset.targeted_fractional_uncertainty_precision

In [ ]:
a = subset.get_uncertainty('teq')
b = subset.get_uncertainty('teq')
average_uncertainty = 0.5*(a + b)
fractional_uncertainty_on_uncertainty = np.abs(a - b)/average_uncertainty
average_uncertainty, fractional_uncertainty_on_uncertainty

If we want to improve the fractional uncertainty, we can update the secret variable `.targeted_fractional_uncertainty_precision` to target a lower value. Here, let's aim for 1% fractional precision. Calculating uncertainties will now take longer because we need to perform more iterations. 

In [ ]:
subset.targeted_fractional_uncertainty_precision = 0.01

In [ ]:
a = subset.get_uncertainty('teq')
b = subset.get_uncertainty('teq')
average_uncertainty = 0.5*(a + b)
fractional_uncertainty_on_uncertainty = np.abs(a - b)/average_uncertainty
average_uncertainty, fractional_uncertainty_on_uncertainty